# Démonstration MOSAIKS : Identification de l'exploitation minière artisanale et à petite échelle en Sierra LeoneCe cahier démontre l’utilisation de l’apprentissage automatique et de l’imagerie satellite pour identifier les sites potentiels d’exploitation minière artisanale et à petite échelle (ASM) en Sierra Leone.

## MéthodologieLe processus comprend les étapes suivantes :1. **Acquisition de données :** des données d'étiquetage sur les sites ASM connus sont acquises. Des images satellite à haute résolution sont utilisées pour extraire les caractéristiques à analyser.2. **Prétraitement des données :** les données d'étiquettes et de caractéristiques sont prétraitées et nettoyées. Cela implique le filtrage des étiquettes à haute fiabilité, la suppression des types de mines non pertinents et la garantie d'une couverture d'inspection suffisante. Les caractéristiques sont mises à l'échelle et formatées pour l'entraînement du modèle.3. **Jointure de données :** les ensembles de données d'étiquettes et d'entités sont joints en fonction de l'intersection spatiale, créant ainsi un ensemble de données reliant les emplacements des sites aux entités dérivées des satellites.4. **Formation et évaluation du modèle :** un modèle d'apprentissage automatique, en particulier Ridge Regression avec étalonnage isotonique, est formé sur l'ensemble de données joint pour prédire la probabilité de présence d'ASM. Le modèle est évalué à l'aide de mesures telles que la zone caractéristique de fonctionnement du récepteur sous la courbe (ROC AUC).5. **Visualisation et interprétation :** Les résultats du modèle sont visualisés à l'aide de cartes et de graphiques. Les caractéristiques importantes identifiées par le modèle sont analysées pour comprendre les tendances associées aux sites ASM.

## InstallationCette section décrit les étapes nécessaires à la configuration de l'environnement d'exécution de l'analyse. Elle comprend :1. **Chargement des bibliothèques :** les bibliothèques Python essentielles pour la manipulation des données, la visualisation et l’apprentissage automatique sont chargées.2. **Montage de Google Drive :** votre Google Drive est monté pour accéder aux données du projet stockées sur le cloud.3. **Définition du répertoire Drive :** le chemin spécifique vers le dossier du projet sur votre Google Drive est défini.4. **Création d'un répertoire local :** un répertoire local temporaire est créé dans Colab pour accélérer le traitement des données. Les données sont copiées dans ce répertoire pour améliorer les performances.

### BibliothèquesCe notebook utilise plusieurs bibliothèques Python clés :* **Traitement et analyse des données :** « pandas » et « numpy » fournissent des structures de données et des fonctions fondamentales pour la manipulation et l'analyse des données. « geopandas » étend ces capacités pour travailler avec des données géospatiales.* **Visualisation :** `matplotlib.pyplot` et `seaborn` permettent la création de visualisations statiques et interactives, facilitant l'exploration des données et la présentation des résultats.* **Apprentissage automatique :** `sklearn` propose une suite complète d'outils pour créer et évaluer des modèles d'apprentissage automatique, notamment des algorithmes tels que la régression Ridge et des méthodes de prétraitement des données.

In [ ]:
!pip install mapclassify

In [50]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

from google.colab import drive
from sklearn.linear_model import RidgeClassifier
from sklearn.calibration import IsotonicRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV

In [51]:
import warnings
from scipy.linalg import LinAlgWarning

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.linear_model._ridge")
warnings.filterwarnings("ignore", category=LinAlgWarning, module="sklearn.linear_model._ridge")

### Montage de Google DriveMonter Google Drive dans Colab permet de connecter votre stockage Google Drive à l'environnement Colab. Cela vous permet d'accéder aux fichiers et dossiers stockés dans votre Drive directement dans votre bloc-notes Colab. Considérez cela comme la création d'un raccourci vers votre Drive dans Colab.**Pourquoi est-ce nécessaire ?**Les blocs-notes Colab s'exécutent sur des machines virtuelles temporaires. Le montage de votre Drive vous permet de charger et d'enregistrer des données sur votre stockage personnel, qui persistent même après la fin de la session Colab. Il permet également un accès transparent à des ensembles de données plus volumineux stockés dans votre Drive, qu'il serait difficile de télécharger directement sur Colab.Après avoir monté Google Drive, nous définissons ensuite le chemin d'accès au dossier du projet sur Google Drive. Ce répertoire contient les fichiers de données nécessaires à l'analyse.

In [ ]:
drive.mount('/content/drive')

In [ ]:
drive_directory = os.path.join(
    "/",
    "content",
    "drive",
    "MyDrive",
    "November 2024 Conference",
)
drive_directory

## Créer un répertoire localCette section crée un répertoire local dans l'environnement Colab pour stocker les données du projet. Les fichiers de données sont copiés du répertoire Google Drive vers ce dossier local.**Pourquoi utiliser un annuaire local ?**L'accès aux données localement sur la machine virtuelle Colab (VM) améliore considérablement la vitesse de traitement par rapport à la lecture directe depuis Google Drive. Bien que la copie des données prenne initialement un peu de temps, cela est compensé par les gains de performances lors de l'analyse.

In [ ]:
local_dir = "/content/data/"

os.makedirs(local_dir, exist_ok=True)

files_to_copy = os.path.join(drive_directory, "data")

shutil.copytree(files_to_copy, local_dir, dirs_exist_ok=True)

Ici, nous examinons le contenu du lecteur local pour nous assurer que nous avons copié les fichiers nécessaires.

In [ ]:
!ls -lh /content/data

## Charger les données de l'étiquette ASMCette section se concentre sur le chargement et la préparation des données d'exploitation minière artisanale et à petite échelle (ASM) pour l'analyse. Les données sont lues à partir d'un fichier CSV (`SLE_mines.csv`) situé dans le répertoire du projet. Plusieurs étapes de prétraitement sont ensuite appliquées pour garantir la qualité et la pertinence des données :1. **Chargement initial et sélection des colonnes :** les données d'étiquette sont chargées dans un DataFrame pandas. Des colonnes spécifiques pertinentes pour l'analyse, telles que les coordonnées de localisation, le type de mine, le niveau de confiance et la couverture d'inspection, sont sélectionnées.2. **Filtrage des données :*** Les sites avec une faible couverture d'inspection (moins de 20 %) sont exclus, sauf s'ils sont étiquetés comme positifs pour l'activité ASM. Cela garantit que seuls les sites avec des données d'inspection suffisantes sont pris en compte.* Les sites classés comme mines « commerciales » sont supprimés, l’accent étant mis sur les opérations artisanales et à petite échelle.* Les sites avec un faible niveau de confiance (inférieur à 3) sont filtrés pour conserver les labels de haute qualité.3. **Restructuration des données :** le DataFrame est réinitialisé pour garantir un index contigu après le filtrage.4. **Création d'un GeoDataFrame :** le DataFrame est converti en GeoDataFrame à l'aide de « geopandas ». Cela permet d'effectuer des opérations et des visualisations spatiales en associant chaque site à un point géographique en fonction de sa longitude et de sa latitude. Ce GeoDataFrame est utilisé dans les étapes suivantes pour l'analyse spatiale et la jonction avec des entités satellites.

In [ ]:
labels = pd.read_csv(os.path.join(local_dir, "SLE_mines.csv"))
labels = labels[
    [
        "lon",
        "lat",
        "unique_id",
        "country",
        "sample_type",
        "mine_type",
        "confidence",
        "label",
        "proportion_inspected"
    ]
]
labels

In [ ]:
labels = labels.loc[(labels["proportion_inspected"] >= 0.2) | (labels["label"] == 1)]
labels.shape

In [ ]:
labels = labels.loc[labels["mine_type"] != "commercial"]
labels.shape

In [ ]:
labels = labels.loc[labels["confidence"] >= 3]
labels.shape

In [ ]:
labels = labels.reset_index(drop=True)
labels

In [ ]:
labels.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

### Transformer des étiquettes en un GeoDataFrameDans cette étape, nous convertissons le DataFrame pandas contenant les données d'étiquette ASM en un GeoDataFrame à l'aide de la bibliothèque « geopandas ». Cette transformation est essentielle pour permettre l'analyse et la visualisation spatiales.**Comment cela se fait :**1. **Créer un GeoDataFrame :** nous utilisons la fonction `geopandas.GeoDataFrame()` pour créer un GeoDataFrame à partir du DataFrame pandas existant (`labels`).1. **Définir la géométrie :** Dans la fonction `GeoDataFrame()`, nous spécifions l'argument `geometry`. Cet argument définit la représentation spatiale de chaque point de données. Nous utilisons `geopandas.points_from_xy()` pour créer des géométries de points à partir des colonnes de longitude (`lon`) et de latitude (`lat`) du DataFrame.1. **Définir le système de référence de coordonnées (CRS) :** nous définissons également l'argument « crs » sur « EPSG:4326 ». Cela définit le système de référence de coordonnées utilisé pour les données, garantissant que les informations spatiales sont interprétées correctement. EPSG:4326 est une norme courante pour les coordonnées géographiques.**Pourquoi c'est nécessaire :*** **Opérations spatiales :** les GeoDataFrames nous permettent d'effectuer des opérations spatiales, comme déterminer les étiquettes qui croisent les entités d'imagerie satellite. Cela est essentiel pour relier les emplacements des sites ASM aux entités extraites.* **Visualisation :** les GeoDataFrames peuvent être facilement tracés sur des cartes, ce qui permet de visualiser la répartition spatiale des sites ASM. Cela aide à comprendre les schémas géographiques de l'activité ASM.* **Intégration avec les outils géospatiaux :** les GeoDataFrames sont compatibles avec d'autres outils et bibliothèques géospatiaux, offrant une flexibilité pour une analyse plus approfondie et une intégration avec les flux de travail SIG.En transformant les étiquettes en un GeoDataFrame, nous renforçons notre analyse avec des capacités spatiales, ouvrant la voie à des explorations plus approfondies de l'activité ASM en Sierra Leone.

In [ ]:
labels_gdf = gpd.GeoDataFrame(
    labels,
    geometry=gpd.points_from_xy(labels.lon, labels.lat),
    crs="EPSG:4326"
)
labels_gdf

In [ ]:
labels_gdf.explore(
    column = "mine_type",
    cmap = "Dark2",
    tiles = "CartoDB positron",
    categorical=True
)

## Chargement et traitement des fonctionnalités satelliteCette section se concentre sur le chargement et la préparation des caractéristiques de l'imagerie satellite pour l'analyse. Ces caractéristiques, dérivées d'images satellite à haute résolution, sont essentielles pour identifier les sites potentiels d'ASM.**Chargement des fonctionnalités :*** Les fonctionnalités sont lues à partir d'un fichier Feather (`SLE_features.feather`) stocké dans le répertoire du projet. Feather est un format rapide et efficace pour stocker des dataframes, ce qui le rend idéal pour gérer de grands ensembles de données comme celui-ci.* Les fonctionnalités sont chargées dans un DataFrame pandas. Chaque ligne représente un emplacement spécifique, identifié par un « image_id », et les colonnes contiennent les fonctionnalités convolutionnelles aléatoires extraites.

In [ ]:
features = pd.read_feather(os.path.join(local_dir, "SLE_features.feather"))
features.head()

**Extraction de la latitude et de la longitude :*** L'image_id contient des informations de latitude et de longitude codées dans sa structure.* Pour les utiliser pour l'analyse spatiale, nous extrayons la latitude et la longitude de `image_id`.* Les valeurs extraites sont stockées sous forme de colonnes « lat » et « lon » distinctes dans le DataFrame.* Les fonctionnalités téléchargées depuis l'API incluent déjà la latitude et la longitude

In [65]:
features[['lat_part', 'lon_part']] = features['image_id'].str.split('__', expand=True)
features['lat'] = features['lat_part'].str.replace('lat_', '').str.replace('--', '.').astype(float)
features['lon'] = features['lon_part'].str.replace('lon_', '').str.replace('--', '.').astype(float)
features = features.drop(['lat_part', 'lon_part'], axis=1)

**Création d'un GeoDataFrame :*** Le DataFrame pandas est ensuite transformé en GeoDataFrame à l'aide de la bibliothèque `geopandas`. Cette étape ajoute des capacités spatiales aux données.* Nous créons des géométries de points à partir des colonnes « lat » et « lon », représentant l'emplacement de chaque entité sur une carte.* Le système de référence de coordonnées (CRS) est défini sur « EPSG:4326 », garantissant une interprétation correcte des informations spatiales.* Enfin, un tampon de 0,005 est appliqué à chaque géométrie de point. Cela crée de petits polygones circulaires autour de chaque point, mais nous incluons également l'argument `cap_style=3` qui crée des coins carrés. Cela transforme effectivement notre géométrie de point en un polygone sur le même espace physique que l'imagerie à partir de laquelle les entités ont été calculées.

In [ ]:
features = gpd.GeoDataFrame(
    features,
    geometry=gpd.points_from_xy(features.lon, features.lat),
    crs="EPSG:4326"
  )
features.geometry = features.geometry.buffer(0.005, cap_style=3)
features.head(3)

## Fonctionnalités de l'exemple de tracéCeci termine le chargement et la préparation des entités satellites. Le GeoDataFrame résultant, contenant à la fois les valeurs des entités et les emplacements géographiques, est maintenant prêt à être joint aux étiquettes ASM pour lier les deux ensembles de données en fonction de la proximité spatiale. Cette intégration permettra au modèle d'apprentissage automatique d'associer les caractéristiques dérivées des satellites à la présence ou à l'absence d'ASM.**Parcelle**- Le graphique ici montre une seule couche de caractéristiques convolutionnelles aléatoires- `feature_0` est choisi arbitrairement- Nous ne pouvons pas utiliser la méthode `.explore()` en raison de contraintes liées à la mémoire système sur la version gratuite de Google Colab

In [ ]:
features.plot(column = "feature_0", legend = True)

## Joindre des étiquettes à des fonctionnalitésCette section décrit le processus de jonction des données d'étiquettes ASM avec les caractéristiques d'imagerie satellite. Cette étape est cruciale pour créer un ensemble de données reliant l'emplacement des sites ASM potentiels à leurs caractéristiques correspondantes dérivées des satellites.**1. Jointure spatiale :*** Nous utilisons la fonction `geopandas.sjoin()` pour effectuer une jointure spatiale entre les étiquettes GeoDataFrame (`labels_gdf`) et les entités GeoDataFrame (`features`).* L'argument `how="inner"` garantit que seuls les sites contenant à la fois des informations d'étiquette et de fonctionnalité sont inclus dans l'ensemble de données joint résultant.* L'argument `predicate="intersects"` spécifie que la jointure doit être basée sur une intersection spatiale. Cela signifie qu'une étiquette sera jointe à une entité si leurs géométries se chevauchent.**2. Ensemble de données résultant :*** La jointure spatiale crée un nouveau GeoDataFrame appelé « joined ».* Ce GeoDataFrame contient toutes les colonnes des ensembles de données d'étiquettes et d'entités, ainsi qu'une nouvelle colonne « index_right » indiquant l'index de l'entité correspondante pour chaque étiquette.* Cet ensemble de données jointes est désormais prêt à être utilisé dans les étapes ultérieures de formation et d'évaluation du modèle. Les informations combinées permettent au modèle d'apprendre des modèles entre les caractéristiques de l'imagerie satellite et la présence d'activité ASM.

In [ ]:
joined = gpd.sjoin(labels_gdf, features, how="inner", predicate="intersects")
joined.head()

## Formation et évaluation du modèleCette section décrit le processus de formation d’un modèle d’apprentissage automatique pour prédire la probabilité de présence d’ASM en fonction des caractéristiques de l’imagerie satellite et évaluer ses performances.**1. Préparation des données :*** L'ensemble de données joint est divisé en entités (X) et en variable cible (y).* `X` contient les fonctionnalités d'imagerie satellite, en particulier les colonnes dont les noms commencent par 'feature_'.* `y` contient l'étiquette ASM (1 pour la présence ASM, 0 pour l'absence).* Les données sont ensuite divisées en ensembles d'entraînement et de test à l'aide de `train_test_split` pour évaluer les performances du modèle sur des données invisibles.

In [69]:
feature_cols = [f"feature_{i}" for i in range(4000)]

In [70]:
X = joined[feature_cols].values
y = joined["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1991
)

**2. Sélection et formation du modèle :*** Un modèle de régression de crête avec calibrage isotonique est utilisé pour la prédiction. Ce modèle est choisi pour sa capacité à gérer des données de grande dimension et sa robustesse aux valeurs aberrantes.* **La régression de Ridge** est un modèle linéaire qui ajoute un terme de pénalité à la fonction de perte, empêchant le surajustement.* **L'étalonnage isotonique** ajuste les prédictions du modèle pour améliorer leur précision et leur fiabilité.* Le réglage des hyperparamètres est effectué à l'aide de `GridSearchCV` pour trouver la meilleure valeur pour le paramètre de régularisation (`alpha`) du modèle de régression Ridge. Cela permet d'optimiser les performances du modèle.

In [71]:
alphas = [0.01, 0.1, 1.0, 10.0]
ridge = RidgeClassifier()
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(
    ridge,
    param_grid,
    cv=5,
    scoring='roc_auc',
    verbose=2,
    n_jobs=-1
)


In [ ]:
grid_search.fit(X_train, y_train)

**3. Évaluation du modèle :*** Le modèle formé est évalué à l’aide de l’ensemble de test, qui n’a pas été utilisé pendant la formation.* La principale mesure d'évaluation est la surface caractéristique de fonctionnement du récepteur sous la courbe (ROC AUC). Cette mesure mesure la capacité du modèle à distinguer les cas positifs (présence d'ASM) et négatifs (absence d'ASM).* La courbe ROC est visualisée pour fournir une représentation graphique des performances du modèle sur différents seuils.

In [ ]:
# Get uncalibrated predictions (decision function)
y_pred_uncal = grid_search.decision_function(X_test)

# Fit isotonic calibration
iso_reg = IsotonicRegression(out_of_bounds='clip')
iso_reg.fit(y_pred_uncal, y_test)

# Get calibrated predictions
y_pred_cal = iso_reg.predict(y_pred_uncal)

# Calculate AUC scores
auc_uncal = roc_auc_score(y_test, y_pred_uncal)
auc_cal = roc_auc_score(y_test, y_pred_cal)

print(f"Best alpha: {grid_search.best_params_['alpha']}")
print(f"AUC before calibration: {auc_uncal:.3f}")
print(f"AUC after calibration: {auc_cal:.3f}")

In [ ]:
# Calculate ROC curves
fpr_uncal, tpr_uncal, _ = roc_curve(y_test, y_pred_uncal)
fpr_cal, tpr_cal, _ = roc_curve(y_test, y_pred_cal)

# Create the plot
plt.figure(figsize=(5, 5))

# Plot both curves
plt.plot(fpr_uncal, tpr_uncal, 'b-', label=f'Uncalibrated (AUC = {auc_uncal:.3f})')
plt.plot(fpr_cal, tpr_cal, 'r-', label=f'Calibrated (AUC = {auc_cal:.3f})')

# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', label='Random')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Before and After Calibration')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

# Set the plot limits
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])

plt.tight_layout()
plt.show()

## Prévision de la probabilité d'ASM en Sierra LeoneCette section étend l’analyse en appliquant le modèle d’apprentissage automatique formé à l’ensemble des fonctionnalités, générant des prédictions sur la probabilité de présence de l’ASM à travers la Sierra Leone.**Processus de prédiction :**1. **Application du modèle :** le modèle de régression de crête formé, ainsi que l'étalonnage isotonique, sont utilisés pour prédire la probabilité de présence d'ASM pour chaque emplacement dans l'ensemble de données d'entités. Cela implique l'application du modèle aux entités d'imagerie satellite associées à chaque emplacement.2. **Stockage des prévisions :** les probabilités prédites sont stockées dans une nouvelle colonne nommée « predicted_probability » dans le GeoDataFrame des entités. Cette colonne représente désormais la probabilité estimée par le modèle d'une activité ASM à chaque emplacement.

In [75]:
y_pred_full = grid_search.decision_function(features[feature_cols].values)
y_pred_full_calibrated = iso_reg.predict(y_pred_full)

features['predicted_probability'] = y_pred_full_calibrated
# features

**Visualisation:**- **Carte de probabilité :** Une carte est générée à l'aide de la méthode `plot()` pour visualiser les probabilités prédites dans l'espace à travers la Sierra Leone. Les dégradés de couleurs représentent la probabilité ASM prédite, les zones de probabilité plus élevée étant indiquées par des couleurs plus chaudes (par exemple, rouge, orange) et les zones de probabilité plus faible par des couleurs plus froides (par exemple, bleu, vert).- Cette carte fournit une vue complète des points chauds potentiels de l'ASM dans la région en fonction des prévisions du modèle.**Connaissances:**La carte de probabilité générée est un outil précieux pour identifier et hiérarchiser les domaines nécessitant une enquête plus approfondie ou des interventions potentielles liées à l'ASM. En mettant en évidence visuellement les zones où la probabilité d'ASM est plus élevée, cette analyse soutient les processus de prise de décision visant à atténuer les impacts négatifs ou à exploiter le potentiel économique de l'ASM en Sierra Leone.

In [ ]:
# plt.figure(figsize=(10, 8))
features.plot(
    column="predicted_probability",
    cmap="viridis",
    legend=True,
    legend_kwds={'label': "Predicted ASM Probability"},
    figsize=(10, 10)
)
plt.title("Predicted ASM Probabilities Across Sierra Leone")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

## ConclusionCe carnet de notes a démontré l'utilisation de l'apprentissage automatique et de l'imagerie satellite pour identifier les sites potentiels d'exploitation minière artisanale et à petite échelle (ASM) en Sierra Leone. Le processus impliquait l'acquisition de données, le prétraitement, l'assemblage, la formation et l'évaluation du modèle, ainsi que la visualisation et l'interprétation.Le modèle a montré des résultats prometteurs pour l'identification de l'activité ASM en Sierra Leone. Le modèle calibré est capable de capturer des signaux pertinents à partir d'images satellite. Les caractéristiques mettent généralement en évidence des éléments tels que la terre nue, l'eau, les infrastructures et les changements dans la végétation, qui peuvent indiquer d'éventuelles opérations minières.Cette approche peut être étendue pour analyser l’ASM dans d’autres régions, tester des modèles d’apprentissage automatique alternatifs, étudier des types de mines spécifiques et analyser des sources ou des fonctionnalités de données satellite supplémentaires.**Travaux supplémentaires*** Étudier l’utilisation d’autres modèles d’apprentissage automatique, tels que Random Forest ou Support Vector Machines.* Intégrer d’autres sources de données, telles que des données géologiques ou des indicateurs socio-économiques.* Regardez les points chauds ASM sur la carte de prédiction et validez le signal dans l'imagerie (c'est-à-dire recherchez des mines et créez de nouvelles étiquettes).**Remerciements**Nous reconnaissons les contributions du Projet sur la gouvernance des ressources (PRG) de l'Université de Californie à Los Angeles, du Laboratoire des marchés environnementaux (emLab) de l'Université de Californie à Santa Barbara et du Centre pour une action mondiale efficace (CEGA) de l'Université de Californie à Berkeley pour avoir fourni les données et le soutien à cette analyse.**Clause de non-responsabilité**Les résultats présentés dans ce carnet sont fournis à titre informatif et de démonstration uniquement et ne doivent pas être considérés comme une preuve définitive de l'activité de l'ASM. Des vérifications et des vérifications sur le terrain supplémentaires sont recommandées pour toute prise de décision liée à l'ASM. Les travaux à l'origine de ce carnet sont en cours et devraient être publiés au cours de l'année prochaine.